<a href="https://colab.research.google.com/github/peter000815/lowcode/blob/master/pic_location1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install exifread

In [26]:
from google.colab import files
uploaded = files.upload()

Saving IMG_8262.HEIC to IMG_8262.HEIC


In [53]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jun 20 21:28:36 2021

@author: zsy
"""

import re
import json
import requests
import exifread


# 转换经纬度格式
def latitude_and_longitude_convert_to_decimal_system(*arg):
    """
    经纬度转为小数, param arg:
    :return: 十进制小数
    """
    return float(arg[0]) + ((float(arg[1]) + (float(arg[2].split('/')[0]) / float(arg[2].split('/')[-1]) / 60)) / 60)


# 读取照片的GPS经纬度信息
def find_GPS_image(pic_path):
    GPS = {}
    date = ''
    with open(pic_path, 'rb') as f:
        tags = exifread.process_file(f)
        for tag, value in tags.items():
            # 纬度
            if re.match('GPS GPSLatitudeRef', tag):
                GPS['GPSLatitudeRef'] = str(value)
            # 经度
            elif re.match('GPS GPSLongitudeRef', tag):
                GPS['GPSLongitudeRef'] = str(value)
            # 海拔
            elif re.match('GPS GPSAltitudeRef', tag):
                GPS['GPSAltitudeRef'] = str(value)
            elif re.match('GPS GPSLatitude', tag):
                try:
                    match_result = re.match('\[(\w*),(\w*),(\w.*)/(\w.*)\]', str(value)).groups()
                    GPS['GPSLatitude'] = int(match_result[0]), int(match_result[1]), int(match_result[2])
                except:
                    deg, min, sec = [x.replace(' ', '') for x in str(value)[1:-1].split(',')]
                    GPS['GPSLatitude'] = latitude_and_longitude_convert_to_decimal_system(deg, min, sec)
            elif re.match('GPS GPSLongitude', tag):
                try:
                    match_result = re.match('\[(\w*),(\w*),(\w.*)/(\w.*)\]', str(value)).groups()
                    GPS['GPSLongitude'] = int(match_result[0]), int(match_result[1]), int(match_result[2])
                except:
                    deg, min, sec = [x.replace(' ', '') for x in str(value)[1:-1].split(',')]
                    GPS['GPSLongitude'] = latitude_and_longitude_convert_to_decimal_system(deg, min, sec)
            elif re.match('GPS GPSAltitude', tag):
                GPS['GPSAltitude'] = str(value)
            elif re.match('.*Date.*', tag):
                date = str(value)
    return {'GPS_information': GPS, 'date_information': date}


# 通过baidu Map的API将GPS信息转换成地址
def find_address_from_GPS(GPS):
    """
    使用Geocoding API把经纬度坐标转换为结构化地址。
    :param GPS:
    :return:
    """
    # 调用百度API的ak值，这个可以注册一个百度开发者获得
    secret_key = 'zbLsuDDL4CS2U0M4KezOZZbGUY9iWtVf'
    #6z6QhM0R41dhxC7MOVBXFQ8Sw1Tf8Tjb   v3
    if not GPS['GPS_information']:
        return '该照片无GPS信息'
    lat, lng = GPS['GPS_information']['GPSLatitude'], GPS['GPS_information']['GPSLongitude']
    baidu_map_api = "http://api.map.baidu.com/geocoder/v2/?ak={0}&callback=renderReverse&location={1},{2}s&output=json&pois=0".format(secret_key, lat, lng)
    #'http://api.map.baidu.com/reverse_geocoding/v3/?ak={0}&callback=renderReverse&location={1},{2}s&output=json&pois=0'.format(secret_key, lat, lng)
    #http://api.map.baidu.com/reverse_geocoding/v3/?ak=您的ak&output=json&coordtype=wgs84ll&location=31.225696563611,121.49884033194
    response = requests.get(baidu_map_api)
    content = response.text.replace("renderReverse&&renderReverse(", "")[:-1]
    print(content)
    baidu_map_address = json.loads(content)
    formatted_address = baidu_map_address["result"]["formatted_address"]
    province = baidu_map_address["result"]["addressComponent"]["province"]
    city = baidu_map_address["result"]["addressComponent"]["city"]
    district = baidu_map_address["result"]["addressComponent"]["district"]
    location = baidu_map_address["result"]["sematic_description"]
    return formatted_address, province, city, district, location


if __name__ == '__main__':
    path = r'IMG_8262.HEIC'   # 图片存放路径
    GPS_info = find_GPS_image(pic_path=path)
    address = find_address_from_GPS(GPS=GPS_info)
    print("拍摄时间：" + GPS_info.get("date_information"))
    print('照片拍摄地址:' + str(address))



{"status":0,"result":{"location":{"lng":120.59818333333327,"lat":31.225185976851689},"formatted_address":"江苏省苏州市吴中区一川街","business":"越溪","addressComponent":{"country":"中国","country_code":0,"country_code_iso":"CHN","country_code_iso2":"CN","province":"江苏省","city":"苏州市","city_level":2,"district":"吴中区","town":"","town_code":"","adcode":"320506","street":"一川街","street_number":"","direction":"","distance":""},"pois":[],"roads":[],"poiRegions":[{"direction_desc":"内","name":"苏州工业职业技术学院","tag":"教育培训;高等院校","uid":"04e8faa419d54cebab175afc","distance":"0"}],"sematic_description":"苏州工业职业技术学院内,张宅村东140米","cityCode":224}}
拍摄时间：2021:05:03 21:21:13
照片拍摄地址:('江苏省苏州市吴中区一川街', '江苏省', '苏州市', '吴中区', '苏州工业职业技术学院内,张宅村东140米')
